# Railway Seat Occupancy - SQL Analysis

This notebook demonstrates data analysis using SQL, a critical skill for Data Analysts. We will query the SQLite database directly to extract insights.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
# Connect to Database
conn = sqlite3.connect('../data/railway.db')

### 1. Overall Occupancy Rate
**Query**: Calculate the total booking percentage across all trains.

In [ ]:
query1 = """
SELECT 
    COUNT(*) as Total_Seats,
    SUM(CASE WHEN Status = 'Booked' THEN 1 ELSE 0 END) as Booked_Seats,
    (SUM(CASE WHEN Status = 'Booked' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as Occupancy_Percentage
FROM bookings;
"""
df_overall = pd.read_sql(query1, conn)
df_overall

### 2. Occupancy by Class
**Insight**: Which class has the highest demand?

In [ ]:
query2 = """
SELECT 
    Class,
    COUNT(*) as Total,
    SUM(CASE WHEN Status = 'Booked' THEN 1 ELSE 0 END) as Booked,
    (SUM(CASE WHEN Status = 'Booked' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as Occupancy_Rate
FROM bookings
GROUP BY Class
ORDER BY Occupancy_Rate DESC;
"""
df_class = pd.read_sql(query2, conn)

plt.figure(figsize=(10, 5))
sns.barplot(x='Class', y='Occupancy_Rate', data=df_class, palette='viridis')
plt.title('Occupancy Rate by Class (SQL Analysis)')
plt.ylabel('Occupancy %')
plt.show()

### 3. Quota Usage Analysis
**Insight**: Who are the passengers? (General, Tatkal, Ladies, Senior Citizens)

In [ ]:
query3 = """
SELECT 
    Quota,
    COUNT(*) as Count
FROM bookings
WHERE Status = 'Booked'
GROUP BY Quota
ORDER BY Count DESC;
"""
df_quota = pd.read_sql(query3, conn)

plt.figure(figsize=(8, 8))
plt.pie(df_quota['Count'], labels=df_quota['Quota'], autopct='%1.1f%%', colors=sns.color_palette('pastel'))
plt.title('Distribution of Bookings by Quota')
plt.show()

### 4. Peak Demand Days (Top 5)
**Insight**: Identifying specific dates with the highest traffic.

In [ ]:
query4 = """
SELECT 
    Date,
    (SUM(CASE WHEN Status = 'Booked' THEN 1 ELSE 0 END) * 100.0 / COUNT(*)) as Occupancy_Rate
FROM bookings
GROUP BY Date
ORDER BY Occupancy_Rate DESC
LIMIT 5;
"""
df_peak = pd.read_sql(query4, conn)
df_peak

In [ ]:
conn.close()